## Modules import

In [2]:
import pandas as pd
import numpy as np
import os

## Import data

In [3]:
path = r'C:\Users\david\04-06-2024 Instacart Basket Analysis'

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge_new_columns.pkl'))

## Data Check

In [5]:
ords_prods_merge.shape

(32404859, 21)

In [6]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_ordinal_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,...,reordered,_merge,Unnamed: 0_y,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day
0,0,0,2539329,1,1,2,8,NaN,True,196,...,0,both,195,Soda,77,7,9.0,Mid-range product,Regularly busy,Average Orders
1,1,1,2398795,1,2,3,7,15.0,False,196,...,1,both,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders
2,2,2,473747,1,3,3,12,21.0,False,196,...,1,both,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders
3,3,3,2254736,1,4,4,7,29.0,False,196,...,1,both,195,Soda,77,7,9.0,Mid-range product,Least busy days,Average Orders
4,4,4,431534,1,5,4,15,28.0,False,196,...,1,both,195,Soda,77,7,9.0,Mid-range product,Least busy days,Most orders - Top 5 Hours


# limiting data frame to million rows

In [7]:
df = ords_prods_merge[:1000000]

In [8]:
df.groupby('product_name')

In [9]:
df.groupby('department_id').agg({'order_ordinal_number': ['mean', 'min', 'max']})

order_ordinal_number        
                              mean min max
department_id                             
4                        18.825780   1  99
7                        17.472355   1  99
13                       17.993423   1  99
14                       19.246334   1  99
16                       19.463012   1  99
17                       11.294069   1  98
19                       19.305237   1  99
20                       17.599636   1  99

In [10]:
ords_prods_merge.shape

(32404859, 21)

In [11]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_ordinal_number'].transform(np.max)

C:\Users\david\AppData\Local\Temp\ipykernel_4064\3935925290.py:1: FutureWarning: The provided callable <function max at 0x000001B394D658A0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_ordinal_number'].transform(np.max)


In [12]:
# loc function that assigns labels to users based on loyalty
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [35]:
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_ordinal_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,...,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_product_price,spending_flag,median_days_between_orders,order_frequency
0,0,0,2539329,1,1,2,8,NaN,True,196,...,9.0,Mid-range product,Regularly busy,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,1,1,2398795,1,2,3,7,15.0,False,196,...,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2,2,473747,1,3,3,12,21.0,False,196,...,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,3,3,2254736,1,4,4,7,29.0,False,196,...,9.0,Mid-range product,Least busy days,Average Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,4,4,431534,1,5,4,15,28.0,False,196,...,9.0,Mid-range product,Least busy days,Most orders - Top 5 Hours,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


## Question 5

In [15]:
# uses groupby and aggregate function to show descriptive statistics of each loyalty category
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

## Question 6

In [16]:
# Determines average price of product purchased by each user
ords_prods_merge['mean_product_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\david\AppData\Local\Temp\ipykernel_4064\72723565.py:1: FutureWarning: The provided callable <function mean at 0x000001B394D662A0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['mean_product_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [17]:
ords_prods_merge[['user_id', 'mean_product_price']].head(20)

,user_id,mean_product_price
0,1,6.367797
1,1,6.367797
2,1,6.367797
3,1,6.367797
4,1,6.367797
5,1,6.367797
6,1,6.367797
7,1,6.367797
8,1,6.367797
9,1,6.367797


In [18]:
# identifies which customers are high spenders vs low spenders
ords_prods_merge.loc[ords_prods_merge['mean_product_price'] >= 10, 'spending_flag'] = 'High spender'

In [19]:
ords_prods_merge.loc[ords_prods_merge['mean_product_price'] < 10, 'spending_flag'] = 'Low spender'

In [20]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [21]:
ords_prods_merge[['user_id', 'mean_product_price', 'spending_flag']].head(20)

,user_id,mean_product_price,spending_flag
0,1,6.367797,Low spender
1,1,6.367797,Low spender
2,1,6.367797,Low spender
3,1,6.367797,Low spender
4,1,6.367797,Low spender
5,1,6.367797,Low spender
6,1,6.367797,Low spender
7,1,6.367797,Low spender
8,1,6.367797,Low spender
9,1,6.367797,Low spender


# Question 7

In [24]:
# median days between user for every user
ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\david\AppData\Local\Temp\ipykernel_4064\2644131335.py:1: FutureWarning: The provided callable <function median at 0x000001B394E9DA80> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [25]:
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders']].head(25)

,user_id,days_since_prior_order,median_days_between_orders
0,1,NaN,20.5
1,1,15.0,20.5
2,1,21.0,20.5
3,1,29.0,20.5
4,1,28.0,20.5
5,1,19.0,20.5
6,1,20.0,20.5
7,1,14.0,20.5
8,1,0.0,20.5
9,1,30.0,20.5


In [26]:
# order frequency flag
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] > 20, 'order_frequency'] = 'Non-frequent customer'

In [27]:
ords_prods_merge.loc[(ords_prods_merge['median_days_between_orders'] <= 20) & (ords_prods_merge['median_days_between_orders'] > 10), 'order_frequency'] = 'Regular customer'

In [28]:
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] <= 10, 'order_frequency'] = 'Frequent customer'

In [29]:
ords_prods_merge['order_frequency'].value_counts(dropna = False)

order_frequency
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
nan                             5
Name: count, dtype: int64

In [31]:
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders', 'order_frequency']].head(25)

,user_id,days_since_prior_order,median_days_between_orders,order_frequency
0,1,NaN,20.5,Non-frequent customer
1,1,15.0,20.5,Non-frequent customer
2,1,21.0,20.5,Non-frequent customer
3,1,29.0,20.5,Non-frequent customer
4,1,28.0,20.5,Non-frequent customer
5,1,19.0,20.5,Non-frequent customer
6,1,20.0,20.5,Non-frequent customer
7,1,14.0,20.5,Non-frequent customer
8,1,0.0,20.5,Non-frequent customer
9,1,30.0,20.5,Non-frequent customer


# Data Export

In [34]:
ords_prods_merge.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge_4-8.pkl'))